media pipe

In [1]:
import cv2
import numpy as np
import tensorflow as tf
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import mediapipe as mp
import os
from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score

In [23]:
model_path = "D:\\electrical eng\\Research\\rppg\\khalaj\\selfie_multiclass_256x256.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def preprocess_image(image, input_shape):
    input_height, input_width = input_shape[1:3]
    image_resized = cv2.resize(image, (input_width, input_height))
    image_normalized = image_resized.astype(np.float32) / 255.0
    return np.expand_dims(image_normalized, axis=0)

def postprocess_output(output, image_shape, threshold=3):
    output = output.squeeze()
    output_resized = cv2.resize(output, (image_shape[1], image_shape[0]))
    mask = output_resized > threshold
    return np.stack([mask] * 3, axis=-1)  # Create a 3-channel mask

BG_COLOR = (192, 192, 192)  # gray
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    image = cv2.flip(image, 1)

    

    input_shape = input_details[0]['shape']
    input_image = preprocess_image(image, input_shape)


    interpreter.set_tensor(input_details[0]['index'], input_image)

    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])

    mask = postprocess_output(output_data, image.shape, threshold=3)

    mask1 = mask[:, :,3 ]  # Ensure mask has 3 channels
    mask2 = mask[:, :,2 ] 
    mask = mask1 + mask2

    bg_image = np.zeros(image.shape, dtype=np.uint8)
    bg_image[:] = BG_COLOR
    output_image = np.where(mask, image, bg_image)

    cv2.imshow('TFLite Selfie Segmentation', output_image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

work on a dataset

In [11]:
input_dir = "D:\\electrical eng\\term 6\\dip\\Project\\datasets"
output_dir = "D:\\electrical eng\\term 6\\dip\\Project\\mediapipe-mask"
image_dir = input_dir

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [6]:
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(input_dir, filename)
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Failed to read {image_path}. Skipping.")
            continue

        input_shape = input_details[0]['shape']
        input_image = preprocess_image(image, input_shape)

        interpreter.set_tensor(input_details[0]['index'], input_image)
        interpreter.invoke()

        output_data = interpreter.get_tensor(output_details[0]['index'])
        mask = postprocess_output(output_data, image.shape, threshold=3)

        mask1 = mask[:, :,3 ]  # Ensure mask has 3 channels
        mask2 = mask[:, :,2 ] 
        mask = mask1 + mask2

        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = BG_COLOR
        output_image = np.where(mask, image, bg_image)

        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, output_image)
        print(f"Processed and saved: {output_path}")

print("Processing complete.")

Processed and saved: D:\electrical eng\term 6\dip\Project\mediapipe-mask\skin_dataset (1).jpg
Processed and saved: D:\electrical eng\term 6\dip\Project\mediapipe-mask\skin_dataset (10).jpg
Processed and saved: D:\electrical eng\term 6\dip\Project\mediapipe-mask\skin_dataset (11).jpg
Processed and saved: D:\electrical eng\term 6\dip\Project\mediapipe-mask\skin_dataset (12).jpg
Processed and saved: D:\electrical eng\term 6\dip\Project\mediapipe-mask\skin_dataset (13).jpg
Processed and saved: D:\electrical eng\term 6\dip\Project\mediapipe-mask\skin_dataset (14).jpg
Processed and saved: D:\electrical eng\term 6\dip\Project\mediapipe-mask\skin_dataset (15).jpg
Processed and saved: D:\electrical eng\term 6\dip\Project\mediapipe-mask\skin_dataset (2).jpg
Processed and saved: D:\electrical eng\term 6\dip\Project\mediapipe-mask\skin_dataset (3).jpg
Processed and saved: D:\electrical eng\term 6\dip\Project\mediapipe-mask\skin_dataset (4).jpg
Processed and saved: D:\electrical eng\term 6\dip\Proj

gaussian_blur and histogram_equalization

In [7]:
def apply_histogram_equalization(image):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

def apply_gaussian_blur(image, kernel_size=(5, 5)):
    return cv2.GaussianBlur(image, kernel_size, 0)

def calculate_metrics(pred_mask, true_mask):
    pred_flat = pred_mask.flatten()
    true_flat = true_mask.flatten()

    iou = jaccard_score(true_flat, pred_flat, average='binary')
    precision = precision_score(true_flat, pred_flat, average='binary')
    recall = recall_score(true_flat, pred_flat, average='binary')
    f1 = f1_score(true_flat, pred_flat, average='binary')
    dice = (2 * precision * recall) / (precision + recall)  # Dice coefficient

    return iou, precision, recall, f1, dice

In [18]:
def evaluate_model_on_dataset(image_dir, output_mask_dir):
    os.makedirs(output_mask_dir, exist_ok=True)

    for image_file in os.listdir(image_dir):
        if image_file.endswith('.jpg') or image_file.endswith('.png'):
            image_path = os.path.join(image_dir, image_file)
            mask_filename = os.path.splitext(image_file)[0] + '.png'  # Assuming mask format is .png

            image = cv2.imread(image_path)
            # Apply histogram equalization
            image = apply_histogram_equalization(image)
            # Apply Gaussian blur
            image = apply_gaussian_blur(image)
            
            input_shape = input_details[0]['shape']
            input_image = preprocess_image(image, input_shape)
            
            interpreter.set_tensor(input_details[0]['index'], input_image)
            interpreter.invoke()
            output_data = interpreter.get_tensor(output_details[0]['index'])
            
            pred_mask = postprocess_output(output_data, image[:,:,0].shape, threshold=3)  # Adjust threshold if needed

            pred_mask_1 = pred_mask[:,:,2]
            pred_mask_2 = pred_mask[:,:,3]
            pred_mask = pred_mask_1 + pred_mask_2

            bg_image = np.zeros(image.shape, dtype=np.uint8)
            bg_image[:] = BG_COLOR
            # pred_mask_33 = np.repeat(pred_mask[:, :, np.newaxis], axis=2)   # Expand mask to have 3 channels
            output_image = np.where(pred_mask, image, bg_image)

            output_path = os.path.join(output_mask_dir, image_file)
            cv2.imwrite(output_path, output_image)
            # print(f"Processed and saved: {output_path}")
            


output_mask_dir = "D:\\electrical eng\\term 6\\dip\\Project\\dataset_enhabced_masks" 
evaluate_model_on_dataset(image_dir,  output_mask_dir)

change color space

In [19]:
def apply_skin_color_threshold(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    lower_skin = np.array([0,30, 90], dtype=np.uint8)
    upper_skin = np.array([255, 255, 255], dtype=np.uint8)
    skin_color_mask = cv2.inRange(hsv_image, lower_skin, upper_skin)
    return skin_color_mask

BG_COLOR = (192, 192, 192) 

In [25]:
def evaluate_model_on_dataset(image_dir, output_mask_dir):
    os.makedirs(output_mask_dir, exist_ok=True)

    for image_file in os.listdir(image_dir):
        if image_file.endswith('.jpg') or image_file.endswith('.png') or image_file.endswith('.jpeg'):
            image_path = os.path.join(image_dir, image_file)
            mask_filename = os.path.splitext(image_file)[0] + '.png'  # Assuming mask format is .png
        
            image = cv2.imread(image_path)
            input_shape = input_details[0]['shape']
            input_image = preprocess_image(image, input_shape)
            
            interpreter.set_tensor(input_details[0]['index'], input_image)
            interpreter.invoke()
            output_data = interpreter.get_tensor(output_details[0]['index'])
            
            pred_mask = postprocess_output(output_data, image[:,:,0].shape)
            pred_mask_1 = pred_mask[:,:,2]
            pred_mask_2 = pred_mask[:,:,3]
            pred_mask = pred_mask_1 + pred_mask_2

            # Convert pred_mask to uint8 for bitwise_and operation
            pred_mask = (pred_mask * 255).astype(np.uint8)

            # Apply HSV skin color thresholding
            skin_color_mask = apply_skin_color_threshold(image)
            thresholded_pred_mask = cv2.bitwise_and(pred_mask, pred_mask, mask=skin_color_mask)

            # Save the thresholded mask
            thresholded_output_path = os.path.join(output_mask_dir, mask_filename)
            cv2.imwrite(thresholded_output_path, thresholded_pred_mask)

    
output_mask_dir = "D:\\electrical eng\\term 6\\dip\\Project\\dataset_hsv_mediapipe_masks" 
evaluate_model_on_dataset(image_dir,  output_mask_dir)

combined

In [29]:
def evaluate_model_on_dataset(image_dir,  output_mask_dir):
    os.makedirs(output_mask_dir, exist_ok=True)
    iou_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    dice_scores = []

    for image_file in os.listdir(image_dir):
        if image_file.endswith('.jpg') or image_file.endswith('.png') or image_file.endswith('.jpeg'):
            image_path = os.path.join(image_dir, image_file)
            mask_filename = os.path.splitext(image_file)[0] + '.png'  # Assuming mask format is .png

            
            image = cv2.imread(image_path)

            input_shape = input_details[0]['shape']
            input_image = preprocess_image(image, input_shape)
            
            interpreter.set_tensor(input_details[0]['index'], input_image)
            interpreter.invoke()
            output_data = interpreter.get_tensor(output_details[0]['index'])
            
            pred_mask = postprocess_output(output_data, image[:,:,0].shape, threshold=3)  # Adjust threshold if needed
            pred_mask_1 = pred_mask[:,:,2]
            pred_mask_2 = pred_mask[:,:,3]
            pred_mask = pred_mask_1 + pred_mask_2

            pred_mask = (pred_mask).astype(np.uint8)

            # Apply HSV skin color thresholding
            skin_color_mask = apply_skin_color_threshold(image)
            thresholded_pred_mask = cv2.bitwise_and(pred_mask, pred_mask, mask=skin_color_mask)

            # Calculate the proportion of the mask that is non-zero before and after thresholding
            mask_coverage_before = np.mean(pred_mask)
            mask_coverage_after = np.mean(thresholded_pred_mask)

            if mask_coverage_after < 0.5 * mask_coverage_before:
                final_pred_mask = pred_mask
            else:
                final_pred_mask = thresholded_pred_mask

            # Save the final mask
            final_output_path = os.path.join(output_mask_dir, mask_filename)
            cv2.imwrite(final_output_path, final_pred_mask * 255)  # Ensure mask is binary (0 or 255)


output_mask_dir = "D:\\electrical eng\\term 6\\dip\\Project\\dataset_hsv_combined_mediapipe_masks" 
evaluate_model_on_dataset(image_dir, output_mask_dir)

ycbcr only

In [30]:
# def rgb_to_hsv(image):
#     return cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

def rgb_to_ycbcr(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)

# def create_rgb_mask(image):
#     r, g, b = image[:,:,0], image[:,:,1], image[:,:,2]
#     mask = (r > 95) & (g > 40) & (b > 20) & (r > g) & (r > b) & (np.abs(r - g) > 15)
#     return mask.astype(np.uint8)

# def create_hsv_mask(image):
#     hsv_image = rgb_to_hsv(image)
#     h, s, v = hsv_image[:,:,0], hsv_image[:,:,1], hsv_image[:,:,2]
#     mask = (h >= 0.0) & (h <= 50.0) & (s >= 0.23*255) & (s <= 0.68*255)
#     return mask.astype(np.uint8)

def create_ycbcr_mask(image):
    ycbcr_image = rgb_to_ycbcr(image)
    y, cb, cr = ycbcr_image[:,:,0], ycbcr_image[:,:,1], ycbcr_image[:,:,2]
    mask = (cr > 135) & (cb > 85) & (y > 80) & (cr <= (1.5862*cb)+20) & (cr >= (0.3448*cb)+76.2069) & \
           (cr >= (-4.5652*cb)+234.5652) & (cr <= (-1.15*cb)+301.75) & (cr <= (-2.2857*cb)+432.85)
    return mask.astype(np.uint8)

In [31]:
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def evaluate_model_on_dataset(image_dir, output_ycbcr_dir):
    os.makedirs(output_ycbcr_dir, exist_ok=True)

    for image_file in os.listdir(image_dir):
        if image_file.endswith('.jpg') or image_file.endswith('.png') or image_file.endswith('.jpeg'):
            image_path = os.path.join(image_dir, image_file)
            mask_filename = os.path.splitext(image_file)[0] + '.png'  # Assuming mask format is .png

            image = cv2.imread(image_path)
 
            # Create mask
            ycbcr_mask = create_ycbcr_mask(image)

            # Save mask
            cv2.imwrite(os.path.join(output_ycbcr_dir, mask_filename), ycbcr_mask * 255)



output_mask_dir_ycbcr = "D:\\electrical eng\\term 6\\dip\\Project\\dataset_ycbcr_masks"
evaluate_model_on_dataset(image_dir,output_mask_dir_ycbcr)

ycbcr combined

In [34]:
def preprocess_image(image, input_shape):
    input_height, input_width = input_shape[1:3]
    image_resized = cv2.resize(image, (input_width, input_height))
    image_normalized = image_resized.astype(np.float32) / 255.0
    return np.expand_dims(image_normalized, axis=0)

def evaluate_model_on_dataset(image_dir, output_mask_dir):
    os.makedirs(output_mask_dir, exist_ok=True)

    for image_file in os.listdir(image_dir):
        if image_file.endswith('.jpg') or image_file.endswith('.png') or image_file.endswith('.jpeg'):
            image_path = os.path.join(image_dir, image_file)
            mask_filename = os.path.splitext(image_file)[0] + '.png'  # Assuming mask format is .png

            
            image = cv2.imread(image_path)

            input_shape = input_details[0]['shape']
            input_image = preprocess_image(image, input_shape)
            
            interpreter.set_tensor(input_details[0]['index'], input_image)
            interpreter.invoke()
            output_data = interpreter.get_tensor(output_details[0]['index'])
            
            pred_mask = postprocess_output(output_data, image[:,:,0].shape, threshold=3)  # Adjust threshold if needed
            pred_mask_1 = pred_mask[:,:,2]
            pred_mask_2 = pred_mask[:,:,3]
            pred_mask = pred_mask_1 + pred_mask_2

            pred_mask = (pred_mask).astype(np.uint8)

        
            skin_color_mask = apply_skin_color_threshold(image)
            thresholded_pred_mask = cv2.bitwise_and(pred_mask, pred_mask, mask=skin_color_mask)

            # Calculate the proportion of the mask that is non-zero before and after thresholding
            mask_coverage_before = np.mean(pred_mask)
            mask_coverage_after = np.mean(thresholded_pred_mask)

            if mask_coverage_after < 0.5 * mask_coverage_before:
                final_pred_mask = pred_mask
            else:
                final_pred_mask = thresholded_pred_mask

            # Save the final mask
            final_output_path = os.path.join(output_mask_dir, mask_filename)
            cv2.imwrite(final_output_path, final_pred_mask * 255)  # Ensure mask is binary (0 or 255)


output_mask_dir_ycbcr = "D:\\electrical eng\\term 6\\dip\\Project\\dataset_ycbcr_combined_masks"
evaluate_model_on_dataset(image_dir,output_mask_dir_ycbcr)
